## Настройки ClickHouse

In [ ]:
import utils
import pandas as pd
import requests
import json
from utils import simple_ch_client

CH_HOST_NAME = '' #добавьте хост в формате строки
CH_USER      = '' #добавьте имя пользователя в формате строки
CH_DB_NAME   = 'autoru_appmetrica' #имя базы данных

#-------------------------------------------
CH_PASS      = open('chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

In [2]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

# Sankey Diagram в plotly

Считываем данные. Нам нужен id пользователя и последовательность из нескольких его шагов (возьмем для простоты 7)

In [ ]:
q = f'''
SELECT
    appmetrica_device_id,
    arrayElement(filt_events, 1) as  step_1,
    arrayElement(filt_events, 2) as  step_2,
    arrayElement(filt_events, 3) as  step_3,
    arrayElement(filt_events, 4) as  step_4,
    arrayElement(filt_events, 5) as  step_5,
    arrayElement(filt_events, 6) as  step_6,
    arrayElement(filt_events, 7) as  step_7
FROM
    (SELECT
        appmetrica_device_id,
        groupArray(event_name) as events,
        count(event_name) as cnt_events,
        groupArray(datetime) as times,
        arrayEnumerate(events) as indexes,
        arrayDifference(arrayMap(x -> toUInt64(x), times)) as times_diffs,
        arrayFilter(e, i -> (i = 1) or (events[i - 1] != events[i]) or (times_diffs[i] >= 1800),
                    events, indexes) as filt_events,
        arrayEnumerate(filt_events) AS steps
    FROM 
        (SELECT
            appmetrica_device_id,
            datetime,
            event_name
        FROM {CH_DB_NAME}.raw_appmetrica_auto_data
        ORDER BY appmetrica_device_id,
            datetime)
    GROUP BY appmetrica_device_id
    HAVING length(steps) <= 10)

format TSVWithNames
'''

cjm_df = my_client.get_clickhouse_df(q)

In [ ]:
cjm_df['finish'] = 'finish'
cjm_df = cjm_df.fillna('finish')

Следующий скрипт позволит выбросить редкие последовательности из выборки

In [ ]:
GLOBAL_STEPS = ['step_1', 'step_2', 'step_3', 'step_4',
       'step_5', 'step_6', 'step_7']
def restrict_data(df):
    total_df = df.groupby(GLOBAL_STEPS).count().reset_index()
    total_df = total_df[total_df['finish'] >= 10][GLOBAL_STEPS]
    print ('uniq sequences:', len(total_df))
    return total_df.set_index(GLOBAL_STEPS).join(df.set_index(GLOBAL_STEPS)).reset_index()

In [ ]:
data = restrict_data(cjm_df)

Посмотрим на данные

In [ ]:
print (len(data))
data.head()

Соберем необходимые массивы для построения Sankey диаграммы

In [ ]:
def get_sankey_data(df):
    steps = GLOBAL_STEPS.copy()
    steps.append('finish')
    label = list(df['step_1'].unique())
    label.append('finish')
    source = [] 
    target = []
    value = [] 

    for ind, step in enumerate(steps[:-1]):
        curr_df = df[['appmetrica_device_id', step, steps[ind + 1]]].drop_duplicates().groupby([step, steps[ind + 1]])\
            .count().reset_index()
        curr_df = curr_df[(curr_df[step] != 'finish') | (curr_df[steps[ind + 1]] != 'finish')]

        source.extend(list(map(lambda x: label.index(x), curr_df[step])))
        outgoing = list(filter(lambda x: x not in label, list(curr_df[steps[ind + 1]].unique())))
        label.extend(outgoing)

        target.extend(list(map(lambda x: label.index(x), curr_df[steps[ind + 1]])))
        value.extend(curr_df['appmetrica_device_id']) 

    return label, source, target, value

In [ ]:
label, source, target, value = get_sankey_data(data)

Посмотрим на содержимое

In [ ]:
print (label,'\n')
print (source,'\n')
print (target,'\n')
print (value,'\n')

Рисуем нашу визуализацию

In [ ]:
import plotly
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label,
      color = ['rgba(31, 119, 180, 0.8)', 'rgba(255, 127, 14, 0.8)', 'rgba(44, 160, 44, 0.8)', 'rgba(214, 39, 40, 0.8)']
    ),
    link = dict(
      source = source,
      target = target,
      value =  value
  ))])

fig.update_layout(title_text="Визуализация поведения пользователей", font_size=10)
fig.show()